In [1]:
from fastai.vision import *

In [2]:
! {sys.executable} -m pip install kaggle --upgrade

'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
! mkdir -p ~/.kaggle/
! mv kaggle.json ~/.kaggle/

The syntax of the command is incorrect.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import configparser
config = configparser.ConfigParser()

In [5]:

config.data_path()

AttributeError: 'ConfigParser' object has no attribute 'data_path'

In [ ]:
path = Config.data_path()/'asl'
path.mkdir(parents=True, exist_ok=True)
path

In [ ]:
! kaggle datasets download -d grassknoted/asl-alphabet -p {path}

In [ ]:
! unzip -q -n {path}/asl-alphabet.zip -d {path}

In [ ]:
path = Config.data_path()/'asl'
path

In [ ]:
path.ls()

In [ ]:
(path/'asl_alphabet_train'/'asl_alphabet_train').ls()

In [ ]:
path = path/'asl_alphabet_train'/'asl_alphabet_train'

In [ ]:
path

In [ ]:
ImageDataBunch.from_folder??

In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
learn

In [ ]:
learn.save('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn

In [ ]:
learn.save('stage-2')

In [ ]:
path

In [ ]:
path_test = path/'..'/'..'

In [ ]:
path

In [ ]:
path_test = Path('/storage/asl/asl_alphabet_test/asl_alphabet_test')

In [ ]:
path_test.ls()

In [ ]:
im1 = open_image(path_test/'b1.jpg')

In [ ]:
pred, _, _ = learn.predict(im1)

In [ ]:
pred

In [ ]:
pred, _, _ = learn.predict(open_image(path_test/'C_test.jpg'))

In [ ]:
pred

In [ ]:
pred, _, _ = learn.predict(open_image(path_test/'b4.jpg'))
pred

In [ ]:
learn.export()

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(do_flip=False), size=200, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.batch_size

In [ ]:
data

In [ ]:
data.show_batch(rows=10)

In [ ]:
tfms = ([*rand_pad(padding=3, size=28, mode='zeros')], [])

In [ ]:
data = (ImageList.from_folder(path)
        .split_by_rand_pct()
        .label_from_folder()
        .databunch()
        .normalize() )

In [ ]:
data.show_batch(rows=10)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, slice(1e-2))

In [ ]:
pred, pred_idx, output = learn.predict(open_image(path_test/'C_test.jpg'))
pred, pred_idx, output

In [ ]:
learn.save('v2')

In [ ]:
learn.export()

*Version 3*

In [ ]:
from fastai.vision import *

In [ ]:
path = Config.data_path()/'asl'/'asl_alphabet_train'/'asl_alphabet_train'
path_test = Path('/storage/asl/asl_alphabet_test/asl_alphabet_test')

In [ ]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=None, max_warp=None)

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=tfms, size=200, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=10)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy, wd=1e-1)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.predict(open_image(path_test/'C_test.jpg'))

In [ ]:
learn.save('v3')

In [ ]:
learn.export()

Version 4

In [ ]:
from fastai.vision import *

In [ ]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=None, max_warp=None)

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=tfms, size=200, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.show_batch(row=5)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy, wd=1e-1)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle?

In [ ]:
learn.fit_one_cycle(8, slice(3e-2))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.save('v4')

In [ ]:
learn.predict(open_image(path_test/'U_test.jpg'))

In [ ]:
learn.export()

Convert fastai model to ONNX

In [ ]:
from fastai.vision import *
from fastai.callbacks import *
from fastai.metrics import *

In [ ]:
from fastai.vision.learner import cnn_config

In [ ]:
path = Config.data_path()/'asl'/'asl_alphabet_train'/'asl_alphabet_train'
path_test = Path('/storage/asl/asl_alphabet_test/asl_alphabet_test')

In [ ]:
def create_head(nf:int, nc:int, lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5,
                concat_pool:bool=True, bn_final:bool=False):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and about `nc` classes."
    lin_ftrs = [nf, 512, nc] if lin_ftrs is None else [nf] + lin_ftrs + [nc]
    ps = listify(ps)
    if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)
    layers = [pool, nn.Flatten()]
    for ni,no,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], ps, actns):
        layers += bn_drop_lin(ni, no, True, p, actn)
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

def create_cnn_model(base_arch:Callable, nc:int, cut:Union[int,Callable]=None, pretrained:bool=True,
                     lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5, custom_head:Optional[nn.Module]=None,
                     bn_final:bool=False, concat_pool:bool=True):
    "Create custom convnet architecture"
    body = create_body(base_arch, pretrained, cut)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, nc, lin_ftrs, ps=ps, concat_pool=concat_pool, bn_final=bn_final)
    else: head = custom_head
    return nn.Sequential(body, head)

def my_cnn_learner(data:DataBunch, base_arch:Callable, cut:Union[int,Callable]=None, pretrained:bool=True,
                lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5, custom_head:Optional[nn.Module]=None,
                split_on:Optional[SplitFuncOrIdxList]=None, bn_final:bool=False, init=nn.init.kaiming_normal_,
                concat_pool:bool=True, **kwargs:Any)->Learner:
    "Build convnet style learner."
    meta = cnn_config(base_arch)
    model = create_cnn_model(base_arch, data.c, cut, pretrained, lin_ftrs, ps=ps, custom_head=custom_head,
        bn_final=bn_final, concat_pool=concat_pool)
    learn = Learner(data, model, **kwargs)
    learn.split(split_on or meta['split'])
    if pretrained: learn.freeze()
    if init: apply_init(model[1], init)
    return learn

In [ ]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=None, max_warp=None)

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=tfms, size=200, num_workers=4).normalize(imagenet_stats)

In [ ]:
learn = my_cnn_learner(data, models.resnet50, metrics=accuracy, wd=1e-1)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(8, slice(3e-2))

In [ ]:
learn.save('v5')

In [ ]:
learn.load('v5')

In [ ]:
for i in path_test.ls():
    print(i, str(learn.predict(open_image(i))[0]))

In [ ]:
learn.export()

In [ ]:
learn.model.eval()

In [ ]:
x = torch.randn(1, 3, 200, 200, requires_grad=False).cuda()
torch_out = torch.onnx._export(learn.model, x, '/storage/asl/asl_alphabet_train/asl_alphabet_train/models/net.onnx', export_params=True)
# str(path)